<center><h1><strong>tau-data Indonesia</strong></h1></center>
<center><h2><strong><font color="blue">Exploratory Data Analysis-01: Data Preparations </font></strong></h2></center>
<img alt="" src="images/Cover.jpg"/>

<b><center>(C) Taufik Sutanto</center>
<center><h3><font color="blue">https://tau-data.id/eda-01/</font></h3></center>

In [ ]:
# Importing Some Python Modules
import warnings; warnings.simplefilter('ignore')
import scipy, itertools, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

plt.style.use('bmh'); sns.set()

In [ ]:
# Importing CSV data  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
try:
    # Running Locally 
    price = pd.read_csv('data/price.csv')
except:
    # Running in Google Colab
    !mkdir data
    !wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/price.csv
    price = pd.read_csv('data/price.csv')

In [ ]:
# Tipe Datanya : DataFrame (df)
print(type(price))
# Ukuran Data
N, P = price.shape
'baris = ', N, ', Kolom = ', P

In [ ]:
# "Mengintip" beberapa data pertamanya
price.head(7)

In [ ]:
# "Mengintip" beberapa data akhirnya
price.tail(9)

In [ ]:
# chosen at random
price.sample(10)

# Removing a variable(s)

In [ ]:
# perhatikan perintahnya tidak menggunakan tanda "()" ==> Properties 
price.columns

In [ ]:
# Drop kolom pertama karena tidak berguna (hanya index)
price.drop("Observation", axis=1, inplace=True)

# Mengoreksi Tipe variabel

In [ ]:
# tipe data di setiap kolom
# Wajib di periksa apakah tipe datanya sudah tepat?
# Perhatikan df sebagaimana semua variable di Python diperlakukan seperti object
price.info()

In [ ]:
# Merubah tipe data "jika" tidak tepat, contoh:
price.dtypes

In [ ]:
# dataframe types: https://pbpython.com/pandas_dtypes.html
price['Parking'] = price['Parking'].astype('category')
price['City_Category'] = price['City_Category'].astype('category')
price.dtypes

In [ ]:
#mengecek apakah ada duplikat data?
print(price.shape)
price.duplicated().sum()

In [ ]:
#menghapus entri yang memiliki data duplikat 
price.drop_duplicates(inplace=True)
print(price.duplicated().sum()) # no more duplicates
print(price.shape) # re-check by printing data size

# Variable Selection

In [ ]:
# Memilih hanya variable dengan tipe tertentu
price_num = price.select_dtypes(include = ['float64', 'int64'])
price_num.head()
# Perhatikan price_num adalah variable df baru! ... (hati-hati di data yang besar)

In [ ]:
# Memilih hanya variable dengan tipe tertentu
price_cat = price.select_dtypes(include = ['category'])
price_cat.head()

# Dasar Pengolahan variabel Kategorik: Dummy Variable

In [ ]:
df = pd.get_dummies(price['Parking'], prefix='Park')
df.head()

# Menggabungkan dengan data awal (concat)

In [ ]:
df2 = pd.concat([price, df], axis = 1)
df2.head().transpose()

# Memilih Data Secara Manual

In [ ]:
# Choosing some columns manually
X = price[['House_Price','Dist_Market']] 
X[:7]

## Apakah ada kecenderungan perbedaan harga rumah akibat dari tipe tempat parkir?

In [ ]:
p= sns.catplot(x="Parking", y="House_Price", data=price)
# Apa yang bisa dilihat dari hasil ini?

In [ ]:
# Distributions
p = sns.distplot(price['House_Price'], kde=True, rug=True)

In [ ]:
# Misal dengan asumsi data berdistribusi normal
# dan menggunakan 95% confidence interval di sekitar variabel "harga"

df = np.abs(price.House_Price - price.House_Price.mean())<=(2*price.House_Price.std())
# mu-2s<x<mu+2s
print(df.shape)
df.head()

In [ ]:
price2 = price[df] # Data tanpa outliers
print(price2.shape, price.shape)
# Perhatikan disini sengaja data yang telah di remove outliernya 
# disimpan dalam variabel baru "Price2"
# Jika datanya besar hati-hati melakukan hal ini

In [ ]:
# Distributions
p = sns.distplot(price2['House_Price'], kde=True, rug=True)

In [ ]:
p= sns.catplot(x="Parking", y="House_Price", data=price2)
# Apa yang bisa dilihat dari hasil ini?

# Missing Values

In [ ]:
# General Look at the Missing Values
print(price2.isnull().sum())

In [ ]:
set(price2['Parking'])

# Gambaran yang Lebih baik tentang MV terutama di Big Data

In [ ]:
sns.heatmap(price2.isnull(), cbar=False)
plt.title('Heatmap Missing Value')
plt.show()

In [ ]:
(price2.isnull().sum()/len(price2)).to_frame('persentase missing')

In [ ]:
# Simplest solution, if the MV is not a lot
# drop rows with missing values : Ada berbagai cara
X = price.dropna() # jika ada MV minimal satu di salah satu kolom, maka baris di hapus
price2.dropna(how='all') # jika ada MV di semua kolom, maka baris di hapus
price2.dropna(thresh=2) # jika ada MV minimal di salah 2 kolom, maka baris di hapus
price2.dropna(subset=['Dist_Hospital'])[:7] # jika ada MV minimal satu di salah kolom Dist_Hospital
# inplace=True if really really sure
price2.dropna(inplace=True)

In [ ]:
print(price2.isnull().sum())

# Statistika Deskriptif

In [ ]:
price2.describe()

In [ ]:
# Statistika Sederhana dari data "Numerik"-nya
price2.describe(include='all')

In [ ]:
# ini adalah parameter tambahan jika kita juga ingin mendapatkan statistik sederhana seluruh datanya
# (termasuk data kategorik)
price[['Dist_Taxi','Parking']].describe(include='all')

# Distribusi nilai pada setiap variabel kategorik

In [ ]:
# get all unique values of a variable/column
for col in price_cat.columns:
    print(col,': ', set(price[col].unique()))

In [ ]:
price['Parking'].unique()

In [ ]:
# price
# Jika yang dibutuhkan memang hanya nama kolom, maka kita bisa melakukan hal ini untuk menghemat penggunaan memory
numVar = price.select_dtypes(include = ['float64', 'int64']).columns
list(numVar)

In [ ]:
# Distribusi tiap data
price['Parking'].value_counts()
# kita bisa juga visualisasikan informasi ini

# Two-Way Tables (contingency tables)

In [ ]:
CT = pd.crosstab(index=price2["City_Category"], columns=price2["Parking"])
CT

# Data Grouping

In [ ]:
# Slicing DataFrame - Just like query in SQL
price2[price2["City_Category"] == "CAT B"].describe()
# Bisa ditambahkan .drop("Parking", axis=1) untuk menghilangkan kolom dengan single value

# Saving (preprocessed) Data

In [ ]:
# Saving the preprocessed Data for future use/analysis
price2.to_csv("data/price_PreProcessed.csv", encoding='utf8')

# Warming up discussion di Forum:

<ul>
	<li>Outliers deteksi?, Apa yang sebaiknya dilakukan? Mengapa?</li>
	<li>Noise: Apa yang disebut dengan noise? Deteksi? Apa yang harus dilakukan?</li>
	<li>duplikasi (row/column). Apa yang harus dilakukan? Mengapa?</li>
	<li>Variabel yang berkorelasi. Apa yang sebaiknya dilakukan? Mengapa?</li>
	<li>Missing Data. Apa yang sebaiknya dilakukan? Mengapa?</li>
</ul>
